In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
import os
import xml.etree.ElementTree as ET

# Path to the directory containing the XML files
xml_dir = "WebNLG/release_v3.0/en/dev/7triples/"

# Initialize the root element for the merged XML file
merged_root = ET.Element("root")

# Loop through all XML files in the directory
for filename in os.listdir(xml_dir):
    if filename.endswith(".xml"):
        # Parse the XML file into an ElementTree object
        tree = ET.parse(os.path.join(xml_dir, filename))

        # Get the root element of the parsed XML
        root = tree.getroot()

        # Append the root element to the merged XML file
        merged_root.extend(root)

# Create an ElementTree object for the merged XML
merged_tree = ET.ElementTree(merged_root)

# Write the merged XML to a file
merged_tree.write("WebNLG/release_v3.0/en/dev/7triples/dev_7triples.xml")


In [5]:
import xml.etree.ElementTree as ET
import xml.etree.ElementTree as ET
from SPARQLWrapper import SPARQLWrapper, JSON

# set up the SPARQL endpoint for DBpedia
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# load the WebNLG dataset from the XML file
#tree = ET.parse("WebNLG/release_v3.0/en/dev/7triples/dev_7triples.xml")
tree = ET.parse("WebNLG/release_v3.0/en/test/rdf-to-text-generation-test-data-with-refs-en.xml")
root = tree.getroot()

def get_entity_class(entity):
    # construct the SPARQL query to retrieve the class of the entity
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?class ?label WHERE {
        <%s> rdf:type ?class .
        ?class rdfs:subClassOf* owl:Thing .
        ?class rdfs:label ?label .
        FILTER (lang(?label) = "en")
    }
    """ % entity

    # set the SPARQL endpoint and query string
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # execute the query and retrieve the results
    results = sparql.query().convert()
    bindings = results['results']['bindings']

    # extract the class label from the results
    if len(bindings) > 0:
        class_uri = bindings[0]['class']['value']
        class_label = bindings[0]['label']['value']
        return class_label
    else:
        return None

def get_entity_subclass(entity):
    clas = get_entity_class(entity)
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    # construct the SPARQL query to retrieve the class of the entity
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?class ?label WHERE {
        <%s> rdfs:subClassOf ?class .
        ?class rdfs:subClassOf* owl:Thing .
        ?class rdfs:label ?label .
        FILTER (lang(?label) = "en")
    }
    """ % clas

    # set the SPARQL endpoint and query string
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # execute the query and retrieve the results
    results = sparql.query().convert()
    bindings = results['results']['bindings']

    # extract the class label from the results
    if len(bindings) > 0:
        class_uri = bindings[0]['class']['value']
        class_label = bindings[0]['label']['value']
        return class_label
    else:
        return None    


# Initialize an empty list to store the results
results = []

# Iterate over all entry elements
for entry in tree.findall('.//entry'):

    # Initialize an empty dictionary for this entry
    entry_dict = {}

    # Extract the eid attribute and store it as the entry ID
    #entry_dict['Entry ID'] = entry.get('eid')

    # Initialize an empty list to store the otriples for this entry
    otriples = []

    # Iterate over all otriple elements in this entry's originaltripleset
    for otriple in entry.findall('.//originaltripleset/otriple'):

        # Extract the text content of the otriple element and append it to the otriples list
        otriples.append(otriple.text)
    print(otriples)

    # Join the otriples list into a single string, with each triple separated by '-'
    
    entry_dict['story'] = entry.find('.//lex').text
    entry_dict['Instances KG'] = ' - '.join(otriples)
    entities = [triple.split(' | ')[2] for triple in otriples]
    entities.append(str(otriples[0].split(' | ')[0]))
    print(entities)
    entities = [entity.replace(" ", "_") for entity in entities]
    # a = [get_entity_class('http://dbpedia.org/resource/'+entity) for entity in entities if '"' not in entity]
    entry_dict['Types KG'] =  ' - '.join([f"{entity.replace(' ', '_')} | type | {get_entity_class('http://dbpedia.org/resource/'+entity)}" for entity in entities if '"' not in entity])
    print(entry_dict['Types KG'])
    entry_dict['Subclasses KG'] =  ' - '.join([f"{entity.replace(' ', '_')} | subclass | {get_entity_subclass('http://dbpedia.org/resource/'+entity)}" for entity in entities if '"' not in entity])
    print(entry_dict['Subclasses KG'])
    

    # Append the entry dictionary to the results list
    results.append(entry_dict)

# Print the results
print(results[0])


['Estádio_Municipal_Coaracy_da_Mata_Fonseca | location | Arapiraca', 'Agremiação_Sportiva_Arapiraquense | league | Campeonato_Brasileiro_Série_C', 'Campeonato_Brasileiro_Série_C | country | Brazil', 'Agremiação_Sportiva_Arapiraquense | nickname | "\'\'Alvinegro"@en', 'Agremiação_Sportiva_Arapiraquense | ground | Estádio_Municipal_Coaracy_da_Mata_Fonseca']
['Arapiraca', 'Campeonato_Brasileiro_Série_C', 'Brazil', '"\'\'Alvinegro"@en', 'Estádio_Municipal_Coaracy_da_Mata_Fonseca', 'Estádio_Municipal_Coaracy_da_Mata_Fonseca']


QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 6: syntax error at \'<\' before \'soccer\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\n    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\n    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n    SELECT ?class ?label WHERE {\n        <soccer league> rdfs:subClassOf ?class .\n        ?class rdfs:subClassOf* owl:Thing .\n        ?class rdfs:label ?label .\n        FILTER (lang(?label) = "en")\n    }\n    \n'

In [101]:
import json 
with open('Datasets/WebNLG/dev_7triples.json', 'w') as f:
    f.write('[')
    for i in results:
        json.dump(i, f)
        f.write(',')
        f.write('\n')
    f.write('{}')    
    f.write(']')


In [37]:
import xml.etree.ElementTree as ET
from SPARQLWrapper import SPARQLWrapper, JSON

# set up the SPARQL endpoint for DBpedia
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# load the WebNLG dataset from the XML file
tree = ET.parse("WebNLG/release_v3.0/en/train/7triples/Astronaut.xml")
root = tree.getroot()

# define the function to query the class of an entity
def get_entity_class(entity):
    # construct the SPARQL query to retrieve the class of the entity
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?class ?label WHERE {
        <%s> rdf:type ?class .
        ?class rdfs:subClassOf* owl:Thing .
        ?class rdfs:label ?label .
        FILTER (lang(?label) = "en")
    }
    """ % entity

    # set the SPARQL endpoint and query string
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # execute the query and retrieve the results
    results = sparql.query().convert()
    bindings = results['results']['bindings']

    # extract the class label from the results
    if len(bindings) > 0:
        class_uri = bindings[0]['class']['value']
        class_label = bindings[0]['label']['value']
        return class_label
    else:
        return None

# loop over all the entries in the dataset
for entry in root.findall(".//entry"):
    # retrieve the triples of the entry
    triples = []
    for otriple in entry.find(".//originaltripleset").findall("otriple"):
        subject, predicate, obj = otriple.text.split(" | ")
        triples.append((subject.strip(), predicate.strip(), obj.strip()))

    # extract the entities from the triples
    entities = set()
    for triple in triples:
        entities.add(triple[0])
        entities.add(triple[2])
    print(entities)    

    # retrieve the class of each entity
    # for entity in entities:
    #     if '"' not in entity:
    #         entity_class = get_entity_class('http://dbpedia.org/resource/'+entity)
    #         print(f'{entity} | type | {entity_class}')

[f"{entity} | type | {get_entity_class('http://dbpedia.org/resource/'+entity)}" for entity in entities if '"' not in entity]


{'United_States', 'Alan_Bean', '"100305.0"^^<http://dbpedia.org/datatype/minute>', '"UT Austin, B.S. 1955"@en', '1963', 'Test_pilot', '"Retired"@en', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"UT Austin, B.S. 1955"@en', '1963', 'Test_pilot', '"June 1981"@en', 'Apollo_12', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"100305.0"^^<http://dbpedia.org/datatype/minute>', '"UT Austin, B.S. 1955"@en', 'Test_pilot', '"Retired"@en', 'Apollo_12', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"100305.0"^^<http://dbpedia.org/datatype/minute>', '"UT Austin, B.S. 1955"@en', '1963', 'Test_pilot', 'Apollo_12', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"UT Austin, B.S. 1955"@en', 'Test_pilot', '"June 1981"@en', 'Apollo_12', '"1932-03-15"^^xsd:date', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"UT Austin, B.S. 1955"@en', '1963', 'Test_pilot', 'Apollo_12', '"1932-03-15"^^xsd:date', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"100305.0"^^<http://dbpedia.org/datatype/mi

['British_Hong_Kong | type | place',
 'United_States | type | person',
 'William_Anders | type | person',
 '1963 | type | time period',
 'Fighter_pilot | type | person']

In [33]:
import xml.etree.ElementTree as ET
from SPARQLWrapper import SPARQLWrapper, JSON

# Initialize the SPARQL endpoint
endpoint_url = "http://dbpedia.org/sparql"
dbpedia = SPARQLWrapper(endpoint_url)

# Parse the XML file
#tree = ET.parse('webnlg-dataset/data/dev/dev.xml')
root = tree.getroot()

# Initialize a list to store the dictionaries
instances = []

# Iterate over each entry
for entry in root.iter('entry'):
    # Get the entity ID
    eid = entry.attrib['eid']
    
    # Initialize a list to store the OTriples
    otriples = []
    
    # Iterate over each otriple in the originaltripleset
    for otriple in entry.find('originaltripleset'):
        # Append the otriple to the list
        otriples.append(otriple.text)
        
    # Concatenate the otriples into a long string
    otriples_str = '-'.join(otriples)
    
    # Get the entity and class information using the function defined earlier
    entities = [ot.split(' | ')[0] for ot in otriples]
    #types = [get_entity_type(e, dbpedia) for e in entities]
    classes = [get_entity_class('http://dbpedia.org/resource/'+e) for e in entities]
  
    
    # Concatenate the entity, type, and class information into a long string
    types_str = '-'.join([f'{e} - type - {c}' for e,  c in zip(entities, classes)])
    print(types_str)
    
    # Create a dictionary for the entry
    entry_dict = {
        'Instance KG': {
            'ID': eid,
            'OTriples': otriples_str,
            'Types KG': types_str
        }
    }
    
    # Append the entry dictionary to the instances list
    instances.append(entry_dict)
    
# Print the instances list
print(instances)


Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person
Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person
Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person
Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person
Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person
Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - 

KeyboardInterrupt: 

In [15]:
a = get_entity_class('http://dbpedia.org/resource/United_States')
print(a)

person
